In [1]:
#softmax: 在多分类中常用的激活函数，是基于逻辑回归的。
#softplus：softplus(x)=log(1+e^x)，近似生物神经激活函数，最近出现的。
#Relu：近似生物神经激活函数，最近出现的。
#tanh：双曲正切激活函数，也是很常用的。
#sigmoid：S型曲线激活函数，最常用的。
#hard_sigmoid：基于S型激活函数。
#linear：线性激活函数，最简单的

import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        pdf = PdfPages('PCP.pdf')

        plt.figure()

        plt.rcParams['figure.dpi'] = 350

     # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="center right")
        plt.title('Acc-loss curve of AD_PCP')
        pdf.savefig()
        plt.close()
        pdf.close()
        plt.show()
history = LossHistory()

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np

t_x=pd.read_csv('X.csv', header=None, index_col=None)
t_y=pd.read_csv('Y.csv', header=None, index_col=None)

xx=t_x.values

xx_y=t_y.values


xx = np.expand_dims(xx, axis=2)
#xx_y = np.expand_dims(xx_y, axis=2)

In [3]:
xx.shape

(17808, 400, 1)

In [4]:
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from sklearn.model_selection import train_test_split
import keras as K
from keras.layers import LSTM, Bidirectional

def creat_model():
    init = K.initializers.glorot_uniform()
    #scheduler = keras.callbacks.ReduceLROnPlateau(simple_adam, 'max', factor=0.5, patience=3)
    model = K.models.Sequential()


    #model.add(Convolution1D(nb_filter=64, filter_length=1, input_shape=(500, 10)))
    #model.add(Activation('relu'))
    #model.add(Flatten())
    #model.add(Dropout(0.4))
    # 定义卷积层

    # 编译模型
    model = keras.Sequential()

    model.add(Convolution1D(16, 8, strides=1, padding='valid', input_shape=(400, 1), activation="relu"))
   # model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))
    model.add(Dropout(0.2))

    model.add(Convolution1D(32, 16, strides=1, padding='valid',activation="relu"))
 #   model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))
    model.add(Dropout(0.2))

    model.add(Convolution1D(64, 32, strides=1, padding='valid',activation="relu"))
#    model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))
    model.add(Dropout(0.2))

 #   model.add(keras.layers.LSTM(32, return_sequences=True))
    model.add(Bidirectional(LSTM(32, return_sequences=True)))

    model.add(Flatten())

    #model.add(K.layers.Dense(units=400 ,kernel_initializer=init, activation='relu'))
    #model.add(Dropout(0.2))
 #   model.add(K.layers.Dense(units=200 ,kernel_initializer=init, activation='relu'))
#    model.add(Dropout(0.2))
    model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])
    return model

from sklearn.model_selection import StratifiedKFold

def train():
    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
    for train, test in kfold.split(xx, xx_y):
        train_x=xx[train]
        train_y=xx_y[train]
        test_x=xx[test]
        test_y=xx_y[test]
        train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.8, test_size=0.2, random_state=0)
        print(train_x.shape,test_x.shape,val_x.shape)
        model = creat_model()
        max_epochs = 20
        print("Starting training ")
        h = model.fit(train_x, train_y, epochs=max_epochs, batch_size=128,shuffle=True, verbose=1,callbacks=[history],validation_data=(val_x, val_y))
        print("Training finished \n")

        eval = model.evaluate(test_x, test_y, verbose=1)
        print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100) )
        print(model.summary())
        a=eval[1] * 100
        np.save('acc'+str(a),a)
        Z = model.predict(xx)
        pd.DataFrame(Z).to_csv(str(a)+'Z.csv',header=None,index=False)
def train2():
    train_x, test_x, train_y, test_y = train_test_split(xx, xx_y, train_size=0.7, test_size=0.3, random_state=0)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.7, test_size=0.3, random_state=0)
    print(train_x.shape,test_x.shape,val_x.shape)
    model = creat_model()
    max_epochs = 20
    print("Starting training ")
    h = model.fit(train_x, train_y, epochs=max_epochs, batch_size=128,shuffle=True, verbose=1,callbacks=[history],validation_data=(val_x, val_y))
    print("Training finished \n")

    eval = model.evaluate(test_x, test_y, verbose=1)
    print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100) )
    print(model.summary())
 #   Z = model.predict(xx)
#    pd.DataFrame(Z).to_csv('Z.csv',header=None,index=False)
    
if __name__ == '__main__':
    train2()
#    train()


(8725, 400, 1) (5343, 400, 1) (3740, 400, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Starting training 
Instructions for updating:
Use tf.cast instead.
Train on 8725 samples, validate on 3740 samples
Epoch 1/20
8725/8725 [==============================] - 823s 94ms/step - loss: 0.6624 - acc: 0.6276 - val_loss: 0.6480 - val_acc: 0.6380
Epoch 2/20
8725/8725 [==============================] - 786s 90ms/step - loss: 0.6269 - acc: 0.6524 - val_loss: 0.6119 - val_acc: 0.6711
Epoch 3/20
8725/8725 [==============================] - 352s 40ms/step - loss: 0.6076 - acc: 0.6800 - val_loss: 0.6208 - val_acc: 0.6631
Epoch 4/20
8725/8725 [==============================] - 231s 26ms/step - loss: 0.5974 - acc: 0.6832 - val_loss: 0.5939 - val_acc: 0.6906
Epoch 5/20
8725/8725 [==============================] - 153s 17ms/step - loss: 0.5906 - acc: 0.6931 - val

In [5]:
history.loss_plot('epoch')

